<a href="https://colab.research.google.com/github/henriqueburis/SITS-BERT_BU_AGRO/blob/main/SITS_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Artigo:https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9252123

In [ ]:
#!pip install colabcode

In [ ]:
#from colabcode import ColabCode
#ColabCode(port=109493)

In [ ]:
!pip install google-cloud-storage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!git clone https://github.com/VSainteuf/lightweight-temporal-attention-pytorch.git
#!git clone https://github.com/linlei1214/SITS-BERT.git

#!git clone

Cloning into 'SITS-BERT'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 150 (delta 23), reused 118 (delta 9), pack-reused 0
Receiving objects: 100% (150/150), 8.70 MiB | 13.07 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [ ]:
!unzip /content/drive/MyDrive/California-Labeled.zip

Archive:  /content/drive/MyDrive/California-Labeled.zip
   creating: California-Labeled/
  inflating: California-Labeled/Test.csv  
  inflating: California-Labeled/Train.csv  
  inflating: California-Labeled/Validate.csv  


In [ ]:
!mkdir /content/dataset_bert
!mkdir /content/dataset_bert/data
!mkdir /content/SITS-BERT/checkpoints/finetune

In [ ]:
from google.colab import auth
from google.cloud import storage

auth.authenticate_user()
client = storage.Client()
print("Client created using default project: {}".format(client.project))

bucket_name = 'bayer_ia'

path = "/content/dataset_bert/data/"
file_erro = []

with open('/content/cloud_bert.txt', 'r') as arquivo:
    for linha in arquivo:
      linha_buffer1 = linha.strip().replace("gs://bayer_ia/","").replace('"','') #.replace('\\','')
      linha_buffer2 = linha_buffer1.replace('sits_bert/samples/fine_tuning/crop_type_test_1/','')

      try:
        bucket = client.get_bucket(bucket_name)
        blob = bucket.blob(linha_buffer1)
        blob.download_to_filename(path+linha_buffer2)
      except:
        print(linha_buffer2)
        file_erro.append(linha)
      #break

Client created using default project: 
95.csv \


In [ ]:
import csv
import glob
import pandas as pd
from random import shuffle
import numpy as np

#n+n+n = bigO 3n-n/3

dados_combinados_buffer1 = []

arquivos_csv = glob.glob('/content/dataset_bert/data/*.csv')

for arquivo in arquivos_csv:
    with open(arquivo, 'r', newline='') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        dados_csv = list(leitor_csv)
        dados_combinados_buffer1.extend(dados_csv[1:])



In [ ]:
label_= []

for idx, line in enumerate(dados_combinados_buffer1):
  line_buffer1 = line[0].replace("'", "")
  #print(line_buffer1[-1])
  label_.append(line_buffer1[-1])
  #break

In [ ]:
dados_int = [int(x) for x in label_]
import matplotlib.pyplot as plt
print(dados_int)
#plt.bar(range(len(dados_int)), dados_int)
#plt.xticks(range(len(dados_int)), range(1, len(dados_int) + 1))
#plt.show()

In [ ]:
import csv
import glob
import pandas as pd
from random import shuffle
import numpy as np

#n+n+n = bigO 3n-n/3

dados_combinados_buffer1 = []

arquivos_csv = glob.glob('/content/dataset_bert/data/*.csv')

for arquivo in arquivos_csv:
    with open(arquivo, 'r', newline='') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        dados_csv = list(leitor_csv)
        dados_combinados_buffer1.extend(dados_csv[1:])

#sufler
#shuffle(dados_combinados)

max_length = 75
dados_combinados_buffer2 = []

for idx, line in enumerate(dados_combinados_buffer1):
  line_buffer1 = line[0].replace("'", "")
  line_data = line_buffer1.split(',')
  line_data = list(map(float, line_data))

  ts = np.reshape(line_data[:-1], (10 + 1, -1)).T
  ts_origin = np.zeros((max_length, 10))
  if(ts.shape[0] < ts_origin.shape[0]):
    dados_combinados_buffer2.append(dados_combinados_buffer1[idx])
    #print(idx)
    #dados_combinados.pop(idx)


n = len(dados_combinados_buffer2)*20/100

print("Train: ",int(n))
print("Validate: ",(int(n)+1))
print("Test: ", len(dados_combinados_buffer2) - (int(n)+int(n)+1))

dataset = {}

dataset['Train'] = dados_combinados_buffer2[0:int(n)]
dataset['Validate'] = dados_combinados_buffer2[int(n)+1:int(n)+int(n)]
dataset['Test'] = dados_combinados_buffer2[int(n)+int(n)+1:]

arquivo_saida = ['Train.csv','Validate.csv','Test.csv']
path = "/content/dataset_bert/"

for item in arquivo_saida:
  type_ = item.replace(".csv", "")
  with open(path+item, 'w', newline='') as saida_csv:
    escritor_csv = csv.writer(saida_csv)
    escritor_csv.writerows(dataset[type_])
  saida_csv.close()

Train:  62670
Validate:  62671
Test:  188010


Analise dos dados

In [ ]:
import numpy as np

result_model = list()
result_model.append("==========threshold de 75 no parâmetro=================== \n")
result_model.append("============================= \n")

max_length = 75

for idx, line in enumerate(dados_combinados_buffer2):
  line_buffer1 = line[0].replace("'", "")
  #print(idx)
  #print(len(line_buffer1[:-1].split(',')))
  line_data = line_buffer1.split(',')
  #print(line_data)
  line_data = list(map(float, line_data))
  #print(line_data)

  ts = np.reshape(line_data[:-1], (10 + 1, -1)).T
  ts_origin = np.zeros((max_length, 10))
  if(ts.shape[0] > ts_origin.shape[0] ):
    #print("Do arquivo: ",ts.shape, " Parâmetro esperado: ",ts_origin.shape)
    #print(dados_combinados[idx])
    result_model.append("\n")
    result_model.append("Do arquivo: "+str(ts.shape)+" Parâmetro esperado: "+str(ts_origin.shape))
    result_model.append("\n")
    result_model.append(str(dados_combinados_buffer2[idx]))
    result_model.append("\n")

    #break

arquivo = open("dadosAnalisados.txt", "a")
arquivo.writelines(result_model)
arquivo.close()

Finetuning do modelo

In [ ]:
#train original
!python /content/SITS-BERT/code/finetuning.py \
    --file_path '/content/dataset_bert/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --max_length 75 \
    --num_features 10 \
    --num_classes 11 \
    --epochs 100 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1

2023-06-21 16:47:38.922710: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 16:47:40.248215: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading Data sets...
training samples: 62670, validation samples: 62669, testing samples: 188010
Creating Dataloader...
Initialing SITS-BERT...
Loading pre-trained model parameters...
Creating Downstream Task Trainer...
Fine-tuning SITS-BERT...
EP0, train_OA=90.12, train_Kappa=0.842, validate_OA=89.94, validate_Kappa=0.831
EP:0 Model Saved on: /content/SITS-BERT/checkpoints/finetune/checkpoint.tar
EP1, train_OA=97.64, train_Kappa=0.963, validate_OA=89.26, validate_Kappa=0.823
EP2, train_OA=99.19, train_Kappa=0.987, validate_OA=88.12, validate_Kappa=0

In [ ]:
#train original
!python /content/SITS-BERT/code/finetuning.py \
    --file_path '/content/California-Labeled/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --max_length 64 \
    --num_features 10 \
    --num_classes 13 \
    --epochs 100 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1

2023-06-21 19:17:33.534355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 19:17:34.422017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading Data sets...
training samples: 1300, validation samples: 1300, testing samples: 318588
Creating Dataloader...
Initialing SITS-BERT...
Loading pre-trained model parameters...
Creating Downstream Task Trainer...
Fine-tuning SITS-BERT...
EP0, train_OA=26.38, train_Kappa=0.202, validate_OA=43.77, validate_Kappa=0.391
EP:0 Model Saved on: /content/SITS-BERT/checkpoints/finetune/checkpoint.tar
EP1, train_OA=50.77, train_Kappa=0.467, validate_OA=74.38, validate_Kappa=0.723
EP:1 Model Saved on: /content/SITS-BERT/checkpoints/finetune/checkpoint.tar
E

In [ ]:
!python /content/SITS-BERT/code/inference.py \
    --file_path '/content/dataset_bert/data/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --max_length 75 \
    --num_features 10 \
    --num_classes 11 \
    --epochs 100 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1

## Test

In [ ]:
#train original --max_length 64 orig .... opt 69 79 128
!python /content/SITS-BERT/code/finetuning.py \
    --file_path '/content/California-Labeled/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --max_length 128 \
    --num_features 10 \
    --num_classes 13 \
    --epochs 10 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1

In [ ]:
#train original --max_length 64 orig .... opt 69 79 128
!python /content/SITS-BERT/code/finetuning.py \
    --file_path '/content/dataset_/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --max_length 64 \
    --num_features 10 \
    --num_classes 7 \
    --epochs 10 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1

In [ ]:
#finetune experimental
!python /content/SITS-BERT/code/finetuning.py \
    --file_path '/content/datset_bert/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --max_length 64 \
    --num_features 10 \
    --num_classes 13 \
    --epochs 100 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1


2023-06-09 19:47:58.025448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 19:47:58.898438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading Data sets...
training samples: 4388, validation samples: 2845, testing samples: 11394
Creating Dataloader...
Initialing SITS-BERT...
Loading pre-trained model parameters...
Creating Downstream Task Trainer...
Fine-tuning SITS-BERT...
EP0, train_OA=71.22, train_Kappa=0.268, validate_OA=12.34, validate_Kappa=0.060
EP:0 Model Saved on: /content/SITS-BERT/checkpoints/finetune/checkpoint.tar
EP1, train_OA=97.63, train_Kappa=0.939, validate_OA=14.80, validate_Kappa=0.037
EP:1 Model Saved on: /content/SITS-BERT/checkpoints/finetune/checkpoint.tar
EP

In [ ]:
####################### Em desenvolvimento ####################################
!python /content/SITS-BERT/code/inference.py \
    --file_path '/content/California-Labeled/' \
    --finetune_path '/content/SITS-BERT/checkpoints/finetune/' \
    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \
    --max_length 64 \
    --num_features 10 \
    --num_classes 13 \
    --epochs 100 \
    --batch_size 128 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 2e-4 \
    --dropout 0.1

# removido
#    --pretrain_path '/content/SITS-BERT/checkpoints/pretrain/' \

In [ ]:
!python /content/SITS-BERT/code/pretraining.py \
    --dataset_path '../data/California-Unlabeled/Pre-Training-Data.csv' \
    --pretrain_path '../checkpoints/pretrain/' \
    --valid_rate 0.03 \
    --max_length 64 \
    --num_features 10 \
    --epochs 100 \
    --batch_size 256 \
    --hidden_size 256 \
    --layers 3 \
    --attn_heads 8 \
    --learning_rate 1e-4 \
    --warmup_epochs 10 \
    --decay_gamma 0.99 \
    --dropout 0.1 \
    --gradient_clipping 5.0